# Sequential Clothing Image Classifier

In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np


# Transform

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(), # to tensor
    transforms.Normalize((0.5), (0.5)) # normalize
])

# Get Data

In [ ]:
# Datasets
trainset = torchvision.datasets.FashionMNIST(train=True, download=True, 
                                  transform=transform, root='./data')

testset = torchvision.datasets.FashionMNIST(train=False, download=True, 
                                            transform=transform, root='./data')

# data loaders
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=30, 
#                                          shuffle=True, num_workers=2)
#testloader = torch.utils.data.DataLoader(testset, batch_size=30, 
#                                         shuffle=False, num_workers=2)

classes = classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

# Check DataLoader Output

In [ ]:
def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2, 0)))
    

dataiter = iter(trainloader)
images, labels = next(dataiter)

print(f'Batch shape: {images.shape}')

#images and labels
imshow(torchvision.utils.make_grid(images))
print(' | '.join(f'{classes[labels[j]]}' for j in range(len(labels))))
    

# Create model from torch sequential model 

In [ ]:
model = nn.Sequential(
    nn.Flatten(), # Turn [30, 1, 28, 28] into [30, 784]
    nn.Linear(28 * 28, 1024), # First FC layer
    nn.ReLU(), # Activation 
    nn.Linear(1024, 1024),
    nn.ReLU(),
    nn.Linear(1024, 10) # Output layer
)

# Loss and Optimizer

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0)

# Train Model 

In [ ]:
for epoch in range(30):
    running_loss = 0
    print_interval = 400
    for i, data in enumerate(trainloader, 0):
        
        inputs, labels = data
        
        optimizer.zero_grad() 
        
        # forward
        outputs = model(inputs)    
        loss = criterion(outputs, labels)
        
        #back
        loss.backward()
        optimizer.step()
        
        # print loss
        running_loss += loss.item()
        if i % print_interval ==  print_interval - 1:
            avg_loss = running_loss / print_interval
            print(f'[Epoch {epoch + 1}, Batch {i + 1:5d}] loss: {avg_loss:.3f}')
            running_loss = 0.0 

print('finished training')
        

# test Model 

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:

        images, labels = data

        outputs = model(images)
        
        _, predicted = torch.max(outputs.data, 1) # index with highest score
        total += labels.size(0) # count total images procoesses so far
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total 
print(f'Final Accuracy on {total} test images: {accuracy:.1f}%') 
        
        

In [ ]:
def create_model(activation_type):
    act = nn.ReLU() if activation_type == 'ReLU' else nn.Sigmoid()
    
    return nn.Sequential(
        nn.Flatten(),
        nn.Linear(784, 1024),
        act,
        nn.Linear(1024, 1024),
        act,
        nn.Linear(1024, 10)
    )

In [ ]:
batch_sizes = [1, 10, 1000]
learning_rates = [1.0, 0.1, 0.01, 0.001]
activations = ['ReLU', 'Sigmoid']
criterion = torch.nn.CrossEntropyLoss()
results = []

for act in activations:
    for bs in batch_sizes:
        for lr in learning_rates:
            # 1. Create DataLoader with batch_size
            trainloader = torch.utils.data.DataLoader(trainset, batch_size=bs, 
                                                      shuffle=True, num_workers=2)
            testloader = torch.utils.data.DataLoader(testset, batch_size=bs, 
                                                     shuffle=False, num_workers=2)
             
            # 2. Initialize Model with activation
            model = create_model(act)
            
            # 3. Initialize Optimizer with lr
            optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0)
            # 4. Train for X epochs
            for epoch in range(30):
                running_loss = 0
                print_interval = max(1, (60000 // bs) // 5)
                for i, data in enumerate(trainloader, 0):
                    inputs, labels = data
        
                    optimizer.zero_grad() 
        
                     # forward
                    outputs = model(inputs)    
                    loss = criterion(outputs, labels)
        
                    #back
                    loss.backward()
                    optimizer.step()
        
                    # print loss
                    running_loss += loss.item()
                    if i % print_interval ==  print_interval - 1:
                        avg_loss = running_loss / print_interval
                        print(f'[Epoch {epoch + 1}, Batch {i + 1:5d}] loss: {avg_loss:.3f}')
                        running_loss = 0.0 

            print('finished training')
            
            # 5. Record final Accuracy
            correct = 0
            total = 0
            with torch.no_grad():
                for data in testloader:
                    images, labels = data
                    outputs = model(images)
        
                    _, predicted = torch.max(outputs.data, 1) # index with highest score
                    total += labels.size(0) # count total images procoesses so far
                    correct += (predicted == labels).sum().item()

            accuracy = 100 * correct / total 
            results.append({
                'Activation': act,
                'Batch Size': bs,
                'Learning Rate': lr,
                'Accuracy': accuracy
            })


print("\n--- FINAL RESULTS TABLE ---")
print(f"{'Act':<10} | {'BS':<6} | {'LR':<6} | {'Accuracy':<10}")
for r in results:
    print(f"{r['Activation']:<10} | {r['Batch Size']:<6} | {r['Learning Rate']:<6} | {r['Accuracy']:.2f}%")

[Epoch 4, Batch 11600] loss: 0.315
[Epoch 4, Batch 12000] loss: 0.358
[Epoch 4, Batch 12400] loss: 0.308
[Epoch 4, Batch 12800] loss: 0.302
[Epoch 4, Batch 13200] loss: 0.354
[Epoch 4, Batch 13600] loss: 0.327
[Epoch 4, Batch 14000] loss: 0.236
[Epoch 4, Batch 14400] loss: 0.343
[Epoch 4, Batch 14800] loss: 0.316
[Epoch 4, Batch 15200] loss: 0.331
[Epoch 4, Batch 15600] loss: 0.280
[Epoch 4, Batch 16000] loss: 0.317
[Epoch 4, Batch 16400] loss: 0.329
[Epoch 4, Batch 16800] loss: 0.350
[Epoch 4, Batch 17200] loss: 0.326
[Epoch 4, Batch 17600] loss: 0.346
[Epoch 4, Batch 18000] loss: 0.255
[Epoch 4, Batch 18400] loss: 0.365
[Epoch 4, Batch 18800] loss: 0.339
[Epoch 4, Batch 19200] loss: 0.257
[Epoch 4, Batch 19600] loss: 0.330
[Epoch 4, Batch 20000] loss: 0.230
[Epoch 4, Batch 20400] loss: 0.279
[Epoch 4, Batch 20800] loss: 0.293
[Epoch 4, Batch 21200] loss: 0.272
[Epoch 4, Batch 21600] loss: 0.307
[Epoch 4, Batch 22000] loss: 0.340
[Epoch 4, Batch 22400] loss: 0.310
[Epoch 4, Batch 2280

libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x1069dfca0>
Traceback (most recent call last):
  File "/Users/joelgc/2026/deeplearn440/env/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1664, in __del__
    self._shutdown_workers()
  File "/Users/joelgc/2026/deeplearn440/env/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1628, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/popen_fork.py", line 40, in wait

KeyboardInterrupt: 